In [4]:
# get data
import pandas as pd
url = f'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/04-monitoring/data/results-gpt4o-mini.csv'
df = pd.read_csv(url)
df = df.iloc[:300]

# Q1

In [5]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

/home/codespace/.local/share/virtualenvs/ZoomcampLLM-Iqt_NaeQ/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



/home/codespace/.local/share/virtualenvs/ZoomcampLLM-Iqt_NaeQ/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
answer_llm = df.iloc[0].answer_llm
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [13]:
embeddings = embedding_model.encode(answer_llm)
print(embeddings[0].round(2))

-0.42


# Q2

In [20]:
df.keys()

Index(['answer_llm', 'answer_orig', 'document', 'question', 'course'], dtype='object')

In [27]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [38]:
evaluations = []

In [40]:
for _, rec in df.iterrows():
    emb_answer_llm = embedding_model.encode(rec['answer_llm'])
    emb_answer_orig = embedding_model.encode(rec['answer_orig'])
    
    
    evaluations.append(emb_answer_orig.dot(emb_answer_llm))

In [44]:
import numpy as np
print(np.percentile(evaluations, 75).round(2))

31.67


# Q3

In [45]:
def normalise_vector(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

In [50]:
norm_evaluations = []

for _, rec in df.iterrows():
    emb_answer_llm = embedding_model.encode(rec['answer_llm'])
    emb_answer_orig = embedding_model.encode(rec['answer_orig'])
    
    norm_emb_answer_llm = normalise_vector(emb_answer_llm)
    norm_emb_answer_orig = normalise_vector(emb_answer_orig)
       
    
    norm_evaluations.append(norm_emb_answer_orig.dot(norm_emb_answer_llm))

In [53]:
print(np.percentile(norm_evaluations, 75).round(2))

0.84


# Q4 

In [56]:
rec = df.iloc[10]

In [57]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(rec['answer_llm'], rec['answer_orig'])[0]

rouge-1 - the overlap of unigrams,
rouge-2 - bigrams,
rouge-l - the longest common subsequence

In [59]:
scores['rouge-1']

{'r': 0.45454545454545453, 'p': 0.45454545454545453, 'f': 0.45454544954545456}

# Q5

In [96]:
rec = df.iloc[10]

scores = rouge_scorer.get_scores(rec['answer_llm'], rec['answer_orig'])[0]

In [101]:
avg_f_score = (scores['rouge-1']['f'] + scores['rouge-2']['f'] + scores['rouge-l']['f']) / 3

print(round(avg_f_score, 2))


0.35490034990035496


# Q6

In [68]:
from rouge import Rouge
rouge_scorer = Rouge()

rouge_scores = []

for _, rec in df.iterrows():
    scores = rouge_scorer.get_scores(rec['answer_llm'], rec['answer_orig'])[0]['rouge-2']['f']
    rouge_scores.append(scores)

In [90]:
test_scores = []

for _, rec in df.iterrows():
    test = rouge_scorer.get_scores(rec['answer_llm'], rec['answer_orig'])[0]['rouge-2']['f']
    test_scores.append(test)

In [94]:
np.mean(rouge_scores).round(2)

np.float64(0.21)